In [1]:
import sys
from pathlib import Path
sys.path.append("/Users/watarukawakami/GSoC/DevoGraph")
sys.path.append("/Users/watarukawakami/GSoC/devolearn")

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import torch as th
import devolearn
from pathlib import Path
DATAPATH = Path('/Users/watarukawakami/GSoC/DevoGraph/data')

from copy import deepcopy
from importlib import reload
# import devograph.datasets.datasets as data

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [3]:
from devolearn import cell_membrane_segmentor

In [4]:
# didn't work
# segmentor = cell_membrane_segmentor()
#df = segmentor.predict_from_video(video_path = str(DATAPATH / "EPIC_raw/20081016_alr-1_10A2_3_L1.mov"), centroid_mode = True, save_folder = os.path.join(DATAPATH, "preds"))
#df = segmentor.predict_from_video(video_path = str(DATAPATH / "EPIC_raw/sm01.mov"), centroid_mode = True, save_folder = os.path.join(DATAPATH, "preds"))
#df = segmentor.predict_from_video(video_path = str(DATAPATH / "EPIC_raw/sm01.mov"), centroid_mode = True, save_folder = os.path.join(DATAPATH, "preds"))

### sample_data of devolearn's cell_membrane_segmentor

In [5]:
# worked
"""
sample_data = Path('/Users/watarukawakami/GSoC/DevoGraph/data/devo_learn/sample_data/')
segmentor = cell_membrane_segmentor()
df = segmentor.predict_from_video(video_path = str(sample_data / "videos/seg_sample.mov"), centroid_mode = True, save_folder = os.path.join(DATAPATH, "preds"))
"""

'\nsample_data = Path(\'/Users/watarukawakami/GSoC/DevoGraph/data/devo_learn/sample_data/\')\nsegmentor = cell_membrane_segmentor()\ndf = segmentor.predict_from_video(video_path = str(sample_data / "videos/seg_sample.mov"), centroid_mode = True, save_folder = os.path.join(DATAPATH, "preds"))\n'

In [6]:
# df.to_csv("centroids.csv")

In [10]:
from devolearn import cell_nucleus_segmentor
sample_data = Path('/Users/watarukawakami/GSoC/DevoGraph/data/devo_learn/sample_data/')
segmentor = cell_nucleus_segmentor()
image, centroids = segmentor.predict(image_path = str(sample_data / "images/nucleus_seg_sample.png"), centroid_mode = True)

In [11]:
from devolearn import cell_nucleus_segmentor
segmentor = cell_nucleus_segmentor()
df = segmentor.predict_from_video(video_path = str(DATAPATH / "EPIC_raw/20081016_alr-1_10A2_3_L1.mov"), centroid_mode = True, save_folder = os.path.join(DATAPATH, "preds"))
#plt.imshow(seg_pred)
#plt.show()

saving predictions: 100%|██████████| 236/236 [00:58<00:00,  4.03it/s]


In [12]:
df.to_csv("centroids.csv")